In [1]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.transforms
import matplotlib.cm
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from ipywidgets import interactive

In [63]:
def gs_from_wind(windspeed):
    #Given the magnitude and sign of the wind in the direction 
    #of the intended velocity (signed wind_par),
    #return the adjusted magnitude of the
    #fly's intended velocity
    if -0.8<=windspeed<=4.:
        return 1.6
    elif windspeed<-0.8:
        return windspeed+2.4
    elif windspeed>4.:
        return windspeed-2.4
    
def signed_wind_par_cartesian(wind_par,original_heading):
    #version 1: inputs are in cartesian coordinates
    #Given the component of wind parallel to the intended velocity,
    #(wind_par vector) and the intended velocity vector,
    #return the signed magnitude of the parallel wind 
    #(+ sign = same direction as intended velocity, - sign = opposite)
    #(prep function for gs_from_wind)
    sign = np.sign(np.dot(wind_par,original_heading))
    return sign*np.sqrt(original_heading.dot(original_heading))

def sigmoid(x,x_0,L,y_0,k):
    return (x-x_0)+(L/2)+y_0 - L/(np.exp(-k*(x-x_0))+1)

def speed_sigmoid_func(x):
    x_0a = -0.4
    x_0b = 3.6
    L = 0.8
    k = 4.
    y_0 = 1.6
    output = np.zeros_like(x)
    output[(x>=x_0a)&(x<=x_0b)] = 1.6
    output[x<x_0a] = sigmoid(x[x<x_0a],x_0a,L,y_0,k)
    output[x>x_0b] = sigmoid(x[x>x_0b],x_0b,L,y_0,k)
    return output
        
def signed_wind_par_polar(theta_0,theta,w):
    #Similar to above except start with wind and intended velocity angle/magnitude
    #theta_0: wind angle
    #theta: intended velocity angle
    #w: wind magnitude
    return w*np.cos(theta_0-theta)
def pp_angle_map_array(theta_0,thetas,r_1s,w):
    wind_par_mags = signed_wind_par_polar(theta_0,thetas,w)
    thetas_adjusted = np.copy(thetas)
#     r_1s[wind_par_mags<-0.8] = wind_par_mags[wind_par_mags<-0.8]+2.4
#     r_1s[wind_par_mags>4.] = wind_par_mags[wind_par_mags>4.]-2.4
    r_1s = speed_sigmoid_func(wind_par_mags)
    sign_change_inds = (r_1s<0.)
    thetas_adjusted[sign_change_inds] = (thetas[sign_change_inds]+np.pi)%(2*np.pi)
    return (np.arctan(w*np.sin(theta_0-thetas_adjusted)/np.abs(r_1s))+thetas_adjusted)%(2*np.pi)

# def pp_angle_map_array_0(theta_0,thetas,r_1s,w):
    

def pp_angle_map_array_1(theta_0,thetas,r_1s,w):
#     signed_wind_par_mags = signed_wind_par_polar(theta_0,thetas,w)
    signed_wind_par_mags = w*np.cos(theta_0-thetas)
    thetas_adjusted = np.copy(thetas)
    adjusted_mag = speed_sigmoid_func(signed_wind_par_mags)
  #  sign_change_inds = (adjusted_mag<0.)
   # thetas_adjusted[sign_change_inds] = (thetas[sign_change_inds]+np.pi)%(2*np.pi)
       
#     output = (np.arctan(w*np.sin(theta_0-thetas_adjusted)/np.abs(r_1s))+thetas_adjusted)%(2*np.pi)

        
    intended_heading_unit_vectors = np.vstack((
        np.cos(thetas),np.sin(thetas)))
#     intended_heading_unit_vectors =
    intended_heading_vectors = adjusted_mag*intended_heading_unit_vectors
    w_vec = np.array([w*np.cos(theta_0),w*np.sin(theta_0)])
    wind_par_vec = (np.dot(
        w_vec,intended_heading_unit_vectors))*intended_heading_unit_vectors
    w_perp_vec = w_vec[:,None] - wind_par_vec
    heading_final_vec = intended_heading_vectors+w_perp_vec
    dispersing_speeds = np.sqrt(np.sum(heading_final_vec**2,axis=0))
    output = np.arctan2(heading_final_vec[1],heading_final_vec[0])
  
    return output
    

def pp_angle_map(theta_0,theta,w):
    wind_par_mag = signed_wind_par_polar(theta_0,theta,w)
    r_1 = gs_from_wind(wind_par_mag)
    if r_1<0:
        theta = (theta+np.pi)%(2*np.pi)
    return (np.arctan(w*np.sin(theta_0-theta)/np.abs(r_1))+theta)%(2*np.pi)



In [64]:
def f(w=1.6,theta_d=1.5):#1.*np.pi/4.):
    n = 10000 
#     n=10
    theta_0 = np.arctan(-1.)
    theta_0 = 7*np.pi/8.
    thetas = np.linspace(360./n,360,n)*np.pi/180
    r_1s = 1.6*np.ones_like(thetas)
   
    
    plt.figure()
    time = 5*60.
    wind_angle = theta_0
    wind_mag = w
    signed_wind_par_mags = w*np.cos(wind_angle-thetas)
#     print(signed_wind_par_mags) 
    adjusted_mag = speed_sigmoid_func(signed_wind_par_mags)
#     print('-------------')
#     print(adjusted_mag)
    intended_heading_unit_vectors = np.vstack((
        np.cos(thetas),np.sin(thetas)))
#     intended_heading_unit_vectors =
#     print('-------------')
#     print(intended_heading_unit_vectors)
    intended_heading_vectors = adjusted_mag*intended_heading_unit_vectors
    w_vec = np.array([wind_mag*np.cos(wind_angle),wind_mag*np.sin(wind_angle)])
#     print('-------------')
#     print(w_vec)
    wind_par_vec = (np.dot(
        w_vec,intended_heading_unit_vectors))*intended_heading_unit_vectors
#     print('-------------')
#     print(wind_par_vec)
    w_perp_vec = w_vec[:,None] - wind_par_vec
    heading_final_vec = intended_heading_vectors+w_perp_vec
    dispersing_speeds = np.sqrt(np.sum(heading_final_vec**2,axis=0))
    track_heading_angles = np.arctan2(heading_final_vec[1],heading_final_vec[0])

#     output = pp_angle_map_array_1(theta_0,thetas,r_1s,w)
    output = pp_angle_map_array(theta_0,thetas,r_1s,w)
    
    mag = time*dispersing_speeds
    mag = dispersing_speeds
    plt.scatter(mag*np.cos(output),mag*np.sin(output),label='output')
   
    plt.scatter(mag*np.cos(track_heading_angles),mag*np.sin(track_heading_angles),label='track_heading_angles')
    plt.scatter(heading_final_vec[0],heading_final_vec[1],label='heading_final_vec')
    plt.axes().set_aspect('equal', 'datalim')
    plt.legend()
    plt.show()
    

In [65]:
interactive_plot = interactive(f, w=(0, 4.0,0.2),theta_d=(0,2*np.pi,np.pi/8),continuous_update=False)
output = interactive_plot.children[-1]
output.layout.height = '1000px'
interactive_plot

interactive(children=(FloatSlider(value=1.6, description='w', max=4.0, step=0.2), FloatSlider(value=1.5, descr…